In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('student35.csv')
#df = df[df['Train/Valid/Test'] != 'Test']

In [2]:
df_exp = df[df['Exp_or_QM'] == 'Exp']

In [3]:
# stereoisomers are super rare

df_exp[(df_exp.can_smiles_solvent.str.contains('@')) | (df_exp.can_smiles_solute.str.contains('@')) | 
       (df_exp.can_smiles_solvent.str.contains('/')) | (df_exp.can_smiles_solute.str.contains('/')) |
       (df_exp.can_smiles_solvent.str.contains("\\", regex=False)) | 
       (df_exp.can_smiles_solute.str.contains('\\', regex=False)) 
      ]

,can_smiles_solute,can_smiles_solvent,DGsolv,index,Exp_or_QM,Train/Valid/Test,predicted
133,C/C=C/C=C/C,C/C=C/C=C/C,-4.45,486,Exp,Train,-4.517644
193,C=C/C=C/CC,C=C/C=C/CC,-4.22,1372,Exp,Train,-4.110133
236,C/C=C/CCC,C/C=C/CCC,-4.07,494,Exp,Train,-3.955752
240,CCCCCCCC/C=C\CCCCCCCCCCCC(=O)OC,CCCCCCCC/C=C\CCCCCCCCCCCC(=O)OC,-15.50,8289,Exp,Train,-15.250817
242,Cl/C=C\Cl,Cl/C=C\Cl,-4.20,15842,Exp,Train,-4.110231
...,...,...,...,...,...,...,...
839832,CCCOC(=O)/C=C\C(=O)OCCC,CCCOC(=O)/C=C\C(=O)OCCC,-9.74,10016,Exp,Test,-9.025261
839924,CCCCCC/C=C\CCCCCCCC(=O)OC,CCCCCC/C=C\CCCCCCCC(=O)OC,-12.50,7862,Exp,Test,-11.903124
840026,CCC/C=C(/C=O)CC,CCC/C=C(/C=O)CC,-6.80,6434,Exp,Test,-6.741358
840233,C/C=C/c1ccc(OC)cc1,C/C=C/c1ccc(OC)cc1,-8.65,512,Exp,Test,-8.177737


In [4]:
df.can_smiles_solvent.unique()

array(['O', 'CCCCCC', 'CCCCCCCC', ..., 'CC(C)CC(=O)CC(C)C', 'C#CCCCCCC',
       'CCCCC(C)C(C)C'], dtype=object)

In [5]:
# cf. Number of data points for each fold for the reference for k-fold after groupby solvents/solutes.

''''
1048 82893
1048 82888
1048 82886
1047 82882
1047 82877
1047 82874
1047 82871
1047 82868
1047 82865
1047 82861
'''

"'\n1048 82893\n1048 82888\n1048 82886\n1047 82882\n1047 82877\n1047 82874\n1047 82871\n1047 82868\n1047 82865\n1047 82861\n"

In [6]:
from sklearn.model_selection import KFold, train_test_split
from tqdm import tqdm

In [7]:
#groupby solutes
from sklearn.model_selection import KFold
DFs_eachfold = []
for i in range(11):
    new_df = pd.DataFrame(columns = df.columns)
    DFs_eachfold.append(new_df)

#####
solute_smiles = []
sub_df_lengths = []
for solute, sub_df in df.groupby('can_smiles_solute'):
    solute_smiles.append(solute)
    sub_df_lengths.append(len(sub_df))
    
sub_df_lengths = np.array(sub_df_lengths)
    
lengths_argsort = np.flip(np.argsort(sub_df_lengths))

sub_df_lengths = sub_df_lengths[lengths_argsort]
solute_smiles = [solute_smiles[x] for x in lengths_argsort]
#####




def build_df_eachfold(DFs_eachfold, indices_for_fold_split, random_state):
    kfold = KFold(n_splits = 11, shuffle = True, random_state = random_state)
    
    if len(indices_for_fold_split) < 11:
        fold_indices_to_consider = np.random.choice(11, len(indices_for_fold_split))
        
        for j, fold_num in enumerate(fold_indices_to_consider):
            solute_smiles_this_fold = [solute_smiles[indices_for_fold_split[j]]]
            DFs_eachfold[fold_num] = pd.concat([DFs_eachfold[fold_num],
                                                df[df.can_smiles_solute.isin(solute_smiles_this_fold)] ])            
    else:
        for fold_num in range(11):
            solute_indices_this_fold = list(kfold.split(indices_for_fold_split))[fold_num][1]
            
            solute_indices_this_fold = [x + min(indices_for_fold_split) for x in solute_indices_this_fold]
            #print(solute_indices_this_fold)
            
            solute_smiles_this_fold = [solute_smiles[x] for x in solute_indices_this_fold]

            DFs_eachfold[fold_num] = pd.concat([DFs_eachfold[fold_num],
                                                df[df.can_smiles_solute.isin(solute_smiles_this_fold)] ])
        
    return DFs_eachfold
    
random_state = 0
DFs_eachfold = build_df_eachfold(DFs_eachfold, [ i for i, x in enumerate(sub_df_lengths) if x >= 184], random_state)

DFs_eachfold = build_df_eachfold(DFs_eachfold, [ i for i, x in enumerate(sub_df_lengths) if x >= 149 and x < 184], 
                                 random_state)


DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 138 and x < 149], 
                                 random_state)

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 130 and x < 138], 
                                 random_state)

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 126 and x < 130], 
                                 random_state)

for j in tqdm(range(125,38,-1)):
    DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                     [ i for i, x in enumerate(sub_df_lengths) if x == j], 
                                     random_state)    

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 33 and x < 39], 
                                 random_state)

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x == 32], 
                                 random_state)

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 28 and x < 32], 
                                 random_state)

for j in tqdm(range(27,22,-1)):
    DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                     [ i for i, x in enumerate(sub_df_lengths) if x == j], 
                                     random_state)  

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 14 and x < 23], 
                                 random_state)

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 12 and x < 14]  , 
                                 random_state)    

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 3 and x < 12] , 
                                 random_state)

for j in tqdm(range(2,0,-1)):
    DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                     [ i for i, x in enumerate(sub_df_lengths) if x == j], 
                                     random_state) 


for fold_num in range(11):
    print(len(DFs_eachfold[fold_num]))

100%|██████████| 2/2 [00:01<00:00,  1.76it/s]

88047
87053
86795
85934
85692
85797
84704
83390
82522
81699
80876


In [11]:
for fold_num in range(11):
    curr_df = DFs_eachfold[fold_num]
    print(
            len(curr_df[curr_df['Exp_or_QM'] == 'Exp']), len(curr_df[curr_df['Exp_or_QM'] == 'QM'])
         )
    
    curr_df['fold_number'] = fold_num

1254 86793
997 86056
952 85843
882 85052
1219 84473
1214 84583
843 83861
644 82746
1321 81201
1284 80415
1027 79849


In [12]:
pd.concat(DFs_eachfold).to_csv('student35_kfold_groupby_solutes.csv', index = False)

In [13]:
df = pd.read_csv('student35_kfold_groupby_solutes.csv')
df

,can_smiles_solute,can_smiles_solvent,DGsolv,index,Exp_or_QM,Train/Valid/Test,predicted,fold_number
0,C1CCCCC1,CC(C)=O,-3.580000,714,Exp,Train,-3.508272,0
1,C1CCCCC1,O=C(O)C(Cl)Cl,-3.290000,819,Exp,Train,-3.228078,0
2,C1CCCCC1,CCCCCCCCCCCCCCCCI,-3.660000,746,Exp,Train,-3.658119,0
3,C1CCCCC1,OCCOCCO,-2.160000,829,Exp,Train,-2.129833,0
4,C1CCCCC1,CCCCCCCCCCCCCCCC,-4.005000,745,Exp,Train,-3.814345,0
...,...,...,...,...,...,...,...,...
932504,Clc1cc(Cl)c(-c2c(Cl)cccc2Cl)c(Cl)c1,O,-1.960000,16283,Exp,Test,-2.287709,10
932505,CCOCCOC(=O)c1ccccc1C(=O)OCCOCC,O,-13.420000,11265,Exp,Test,-13.112354,10
932506,C/C=C/C#N,C/C=C/C#N,-5.610000,446,Exp,Test,-5.275470,10
932507,CCCCCC/C=C\CCCCCCCC(=O)OC,CCCCCC/C=C\CCCCCCCC(=O)OC,-12.500000,7862,Exp,Test,-11.903124,10
